In [5]:
# import pandas
import pandas as pd

# import plotly
import plotly.express as px
from plotly.subplots import make_subplots

In [6]:
#%%  Reading CSV file into a DataFrame

avg_df = pd.read_csv('files\\avg_df.csv')

# Convert 'timestamp' to datetime format
avg_df['timestamp'] = pd.to_datetime(avg_df['timestamp'].str.split('/').str[0])

# Sort first by symbols (to group all records for each symbol together)
# Then by timestamp (descending) within each symbol group
avg_df = avg_df.sort_values(
    ['symbols', 'timestamp'], 
    ascending=[True, False]
).reset_index(drop=True)


In [15]:
# Create the figure
fig1 = px.line(
    avg_df,
    x='timestamp',
    y='average_price',
    color='symbols',
    custom_data=['symbols', 'average_price'],
    title='Price Growth (Log Scale)',
    labels={
        'timestamp': 'Date',
        'average_price': 'Price (USD)',
        'symbols': 'Cryptocurrency'
    },
    color_discrete_map={
        'BTC/USDT': '#FC922F',  # Bitcoin Orange
        'ETH/USDT': '#626AFF',  # Ethereum Blue
        'BNB/USDT': '#FFCF3D',  # Binance Yellow
        'SOL/USDT': '#CF62E7',   # Solana Pink
        'XRP/USDT': '#DCDCDC',  # Ripple Grey
        'AVAX/USDT': '#FF3A3A',  # Avalanche Red
    },
    markers=False,
    line_shape='spline',
)

# Update traces to reduce marker size
fig1.update_traces(
    line=dict(width= 1.5, smoothing=0.5),
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Price: $%{customdata[1]:,.2f}<br>" +
                  "<extra></extra>"
)

fig1.update_layout(
    template="plotly_dark",              # inherit base dark theme 
        hovermode="closest",
        dragmode="pan",
        title=dict(font=dict(size=14, color='white'), xanchor="left"),
        hoverlabel=dict(font=dict(family="IBM Plex Sans", size=12)),
        font=dict(family="IBM Plex Sans", color="white"),
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.05),
        margin=dict(l=10, r=10, t=40, b=10),
        yaxis=dict(type="log", title="Price (USD)", dtick=1,),
        xaxis=dict(rangeslider=dict(visible=True, thickness=0.1), type="date"),
        height=500, width=705,
)

# Show the plot
fig1.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})


# get the full HTML as a string, with Plotly.js from CDN
fig1html = fig1.to_html(include_plotlyjs="cdn", full_html=True)

# inject the Google Fonts link right after <head>
font_link = '<link href="https://fonts.googleapis.com/css2?family=IBM+Plex+Sans&display=swap" rel="stylesheet">'
fig1html = fig1html.replace("<head>", f"<head>\n    {font_link}")

# save the HTML to a file
with open("plots/html/crypto_price_growth.html", "w") as f:
    f.write(fig1html)

# **Price Normalization**

Each cryptocurrency price was normalized to 100% from July 2020 and then January 2024 to enable direct comparison of growth rates.

In [8]:
#%% Creating Jul2020 Normalized Price Data

# Create a copy of the DataFrame to avoid modifying the original
normalized_df_2020 = avg_df.copy()

# Filter data starting from January 2024
start_date = pd.to_datetime('2020-07-01')
normalized_df_2020 = normalized_df_2020[normalized_df_2020['timestamp'] >= start_date]

# Sort by timestamp ascending to get correct base prices
normalized_df_2020 = normalized_df_2020.sort_values(['symbols', 'timestamp'], ascending=[True, True])

# Get the first value for each symbol
base_values = normalized_df_2020.groupby('symbols').first()['average_price']

# Calculate normalized prices 
normalized_df_2020['normalized_price'] = normalized_df_2020.apply(
    lambda x: (x['average_price'] / base_values[x['symbols']]) * 100,
    axis=1
)

In [16]:
#%% Create the normalized price evolution graph

# Create the figure
fig2 = px.line(
    normalized_df_2020,
    x='timestamp',
    y='normalized_price',
    color='symbols',
    custom_data=['symbols', 'average_price'],
    title='Normalized Price - Since July 2020',
    labels={
        'timestamp': 'Date',
        'normalized_price': 'Price Change (%)',
        'symbols': 'Cryptocurrency'
    },
    color_discrete_map={
        'BTC/USDT': '#FC922F',  # Bitcoin Orange
        'ETH/USDT': '#626AFF',  # Ethereum Blue
        'BNB/USDT': '#FFCF3D',  # Binance Yellow
        'SOL/USDT': '#CF62E7',   # Solana Pink
        'XRP/USDT': '#DCDCDC',  # Ripple Grey
        'AVAX/USDT': '#FF3A3A',  # Avalanche Red
    },
    markers=False,
    line_shape='spline'
)

# Update traces to reduce marker size
fig2.update_traces(
    line=dict(width= 1.5, smoothing=0.5),
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Change: %{y:.1f}%<br>" +
                  "Price: $%{customdata[1]:,.2f}<br>" +
                  "<extra></extra>",
)   

# Customize the layout
fig2.update_layout(
    template="plotly_dark",              # inherit base dark theme 
    hovermode="closest",
    dragmode="pan",
    title=dict(font=dict(size=14, color='white'), xanchor="left"),
    hoverlabel=dict(font=dict(family="IBM Plex Sans", size=12)),
    font=dict(family="IBM Plex Sans", color="white"),
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.05),
    margin=dict(l=10, r=10, t=40, b=10),
    yaxis=dict(title="Price Change (%)", ticksuffix='%', tickformat=",d"),
    xaxis=dict(rangeslider=dict(visible=True, thickness=0.1), type="date"),
    height=500, width=705,
)

# Show the plot
fig2.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})


# get the full HTML as a string, with Plotly.js from CDN
fig2html = fig2.to_html(include_plotlyjs="cdn", full_html=True)

# inject the Google Fonts link right after <head>
font_link = '<link href="https://fonts.googleapis.com/css2?family=IBM+Plex+Sans&display=swap" rel="stylesheet">'
fig2html = fig2html.replace("<head>", f"<head>\n    {font_link}")

# save the HTML to a file
with open("plots/html/jul2020_crypto_relative_price_performance.html", "w") as f:
    f.write(fig2html)

In [11]:
#%% Creating Jan2024 Normalized Price Data

# Create a copy of the DataFrame to avoid modifying the original
normalized_df_2024 = avg_df.copy()

# Filter data starting from January 2024
start_date = pd.to_datetime('2024-01-01')
normalized_df_2024 = normalized_df_2024[normalized_df_2024['timestamp'] >= start_date]

# Sort by timestamp ascending to get correct base prices
normalized_df_2024 = normalized_df_2024.sort_values(['symbols', 'timestamp'], ascending=[True, True])

# Get the first value for each symbol
base_values = normalized_df_2024.groupby('symbols').first()['average_price']

# Calculate normalized prices 
normalized_df_2024['normalized_price'] = normalized_df_2024.apply(
    lambda x: (x['average_price'] / base_values[x['symbols']]) * 100,
    axis=1
)

In [17]:
#%% Create the normalized price evolution graph

# Create the figure
fig3 = px.line(
    normalized_df_2024,
    x='timestamp',
    y='normalized_price',
    color='symbols',
    custom_data=['symbols', 'average_price'],
    title='Normalized Price - Since January 2024',
    labels={
        'timestamp': 'Date',
        'normalized_price': 'Price Change (%)',
        'symbols': 'Cryptocurrency'
    },
    color_discrete_map={
        'BTC/USDT': '#FC922F',  # Bitcoin Orange
        'ETH/USDT': '#626AFF',  # Ethereum Blue
        'BNB/USDT': '#FFCF3D',  # Binance Yellow
        'SOL/USDT': '#CF62E7',   # Solana Pink
        'XRP/USDT': '#DCDCDC',  # Ripple Grey
        'AVAX/USDT': '#FF3A3A',  # Avalanche Red
    },
    markers=True,
    line_shape='spline'
    
)

# Update traces to reduce marker size
fig3.update_traces(
    marker=dict(size=3),
    line=dict(width= 1.5, smoothing=1),
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Change: %{y:.1f}%<br>" +
                  "Price: $%{customdata[1]:,.2f}<br>" +
                  "<extra></extra>",
) 

# Customize the layout
fig3.update_layout(
    template="plotly_dark",              # inherit base dark theme 
    hovermode="closest",
    dragmode="pan",
    title=dict(font=dict(size=14, color='white'), xanchor="left"),
    hoverlabel=dict(font=dict(family="IBM Plex Sans", size=12)),
    font=dict(family="IBM Plex Sans", color="white"),
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.05),
    margin=dict(l=10, r=10, t=40, b=10),
    yaxis=dict(title="Price Change (%)", ticksuffix='%', tickformat=",d"),
    xaxis=dict(rangeslider=dict(visible=True, thickness=0.1), type="date"),
    height=500, width=705,
)

# Show the plot
fig3.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})


# get the full HTML as a string, with Plotly.js from CDN
fig3html = fig3.to_html(include_plotlyjs="cdn", full_html=True)
# inject the Google Fonts link right after <head>
font_link = '<link href="https://fonts.googleapis.com/css2?family=IBM+Plex+Sans&display=swap" rel="stylesheet">'
fig3html = fig3html.replace("<head>", f"<head>\n    {font_link}")

with open("plots/html/jan2024_crypto_relative_price_performance.html", "w") as f:
    f.write(fig3html)

In [13]:
#%% Creating Jan2025 Normalized Price Data

# Create a copy of the DataFrame to avoid modifying the original
normalized_df_2025 = avg_df.copy()

# Filter data starting from January 2024
start_date = pd.to_datetime('2025-01-01')
normalized_df_2025 = normalized_df_2025[normalized_df_2025['timestamp'] >= start_date]

# Sort by timestamp ascending to get correct base prices
normalized_df_2025 = normalized_df_2025.sort_values(['symbols', 'timestamp'], ascending=[True, True])

# Get the first value for each symbol
base_values = normalized_df_2025.groupby('symbols').first()['average_price']

# Calculate normalized prices 
normalized_df_2025['normalized_price'] = normalized_df_2025.apply(
    lambda x: (x['average_price'] / base_values[x['symbols']]) * 100,
    axis=1
)

In [18]:
#%% Create the normalized price evolution graph

# Create the figure
fig4 = px.line(
    normalized_df_2025,
    x='timestamp',
    y='normalized_price',
    color='symbols',
    custom_data=['symbols', 'average_price'],
    title='Normalized Price - Since January 2025',
    labels={
        'timestamp': 'Date',
        'normalized_price': 'Price Change (%)',
        'symbols': 'Cryptocurrency'
    },
    color_discrete_map={
        'BTC/USDT': '#FC922F',  # Bitcoin Orange
        'ETH/USDT': '#626AFF',  # Ethereum Blue
        'BNB/USDT': '#FFCF3D',  # Binance Yellow
        'SOL/USDT': '#CF62E7',   # Solana Pink
        'XRP/USDT': '#DCDCDC',  # Ripple Grey
        'AVAX/USDT': '#FF3A3A',  # Avalanche Red
    },
    markers=True,
    line_shape='spline'
)

# Update traces to reduce marker size
fig4.update_traces(
    marker=dict(size=4),
    line=dict(width= 1.5, smoothing=1),
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Change: %{y:.1f}%<br>" +
                  "Price: $%{customdata[1]:,.2f}<br>" +
                  "<extra></extra>",
) 

# Customize the layout
fig4.update_layout(
    template="plotly_dark",              # inherit base dark theme 
    hovermode="closest",
    dragmode="pan",
    title=dict(font=dict(size=14, color='white'), xanchor="left"),
    hoverlabel=dict(font=dict(family="IBM Plex Sans", size=12)),
    font=dict(family="IBM Plex Sans", color="white"),
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.05),
    margin=dict(l=10, r=10, t=40, b=10),
    yaxis=dict(title="Price Change (%)", ticksuffix='%', tickformat=",d"),
    xaxis=dict(rangeslider=dict(visible=True, thickness=0.1), type="date"),
    height=500, width=705,
)

# Show the plot
fig4.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

# get the full HTML as a string, with Plotly.js from CDN
fig4html = fig4.to_html(include_plotlyjs="cdn", full_html=True)
# inject the Google Fonts link right after <head>
font_link = '<link href="https://fonts.googleapis.com/css2?family=IBM+Plex+Sans&display=swap" rel="stylesheet">'
fig4html = fig4html.replace("<head>", f"<head>\n    {font_link}")

with open("plots/html/jan2025_crypto_relative_price_performance.html", "w") as f:
    f.write(fig4html)